In [18]:
from tensorflow import keras
import numpy as np
import pandas as pd
import tensorflow.keras.layers as layers
from sklearn.model_selection import KFold

import training_utils

# Configuracion 

In [19]:
img_size=(240, 320)
img_channels = 3
batch_size=32
epochs = 50
display_epochs = (0, 100)
inputs = keras.Input(shape= img_size + (img_channels,))
debug = False

In [20]:
dataset_path = '/Users/olove/Library/CloudStorage/OneDrive-Personal/AI datasets/CrowdCounter'


# Carga de los datos


In [21]:
labels_df = pd.read_csv(dataset_path + '/labels.csv')
labels_df['image_name'] = labels_df['id'].map('seq_{:06d}.jpg'.format)
labels_df.drop("id", axis=1,inplace=True)
labels_df = labels_df.sample(frac=1).reset_index(drop=True)
display(labels_df)

,count,image_name
0,31,seq_001125.jpg
1,36,seq_000247.jpg
2,22,seq_000887.jpg
3,47,seq_000709.jpg
4,29,seq_001436.jpg
...,...,...
1995,29,seq_001973.jpg
1996,24,seq_000010.jpg
1997,30,seq_001115.jpg
1998,30,seq_001941.jpg


# Introducción



## Baseline sencillo

Durante este entrenamiento vamos a estar utilizando el mae para medir la calidad de los modelos.
Pera ver si nuestros modelos realmente estan aprendiendo vamos a utilizar un baseline sencillo que consiste en predecir la media de la distribución de los datos de entrenamiento.

In [24]:
baseline_mae = np.mean(np.abs(labels_df['count'] - np.mean(labels_df['count'])))
print(f'Baseline MAE: {baseline_mae}')

Baseline MAE: 5.5322575


Con esto podemos ver que una solución sencilla tiene un MAE de 5.53. Por lo tanto cualquier modelo que no supere este valor no estará aprendiendo una solución util a este problema.

# Procedimiento de evaluación de los modelos

Para evaluar los modelos se va a utilizar una validación k-fold. 
Esto es debido a que el dataset es pequeño y al entrenar se aprecia que hay mucha variablilidad en el resultado de la validación.



In [ ]:
def k_fold_validation(i_model, model_filename):

    callbacks_list = [
        #    keras.callbacks.EarlyStopping(
        #        monitor="val_loss", patience=4
        #    ),
        keras.callbacks.ModelCheckpoint(
            filepath=model_filename,
            monitor="val_loss",
            save_best_only=True
        ),
        #    keras.callbacks.TensorBoard()
    ]
    
    # Guradamos los pesos iniciales del modelo para poder reiniciarlos en cada iteración sin tener que re compilar/contruir el modelo
    Wsave = i_model.get_weights()

    i_kf = KFold(n_splits = 5, shuffle = True, random_state = 2)

    history_store = []

    for i_result in i_kf.split(labels_df):

        train = labels_df.iloc[i_result[0]]
        test =  labels_df.iloc[i_result[1]]

        if debug:
            print(i_result[0])
            print(i_result[1])

            display(labels_df)
            display(train)
            print(f'Train size: {len(train)}')
            display(test)
            print(f'Test size: {len(test)}')

        (train_generator, validation_generator) = training_utils.load_generators(train, test, dataset_path, batch_size, img_size)

        i_model.set_weights(Wsave)
        i_history = i_model.fit(train_generator,
                                epochs=epochs,
                                callbacks = callbacks_list,
                                validation_data=validation_generator,
                                )

        history_store.append(i_history)

    return history_store
    

# Los Modelos

## Modelos convulucionales simples

Como primer modelo intentamos eperimentar con modelos convulucionales simples.
Para ellos utilizaremos unicamente unas capas convulucionales y unas capas densas sin nungun otro añadido.

Con esto pretendemos ver si un modelo convulucional puede aprender o superar nuestra solución baseline.

In [ ]:
def basic_covnet_block():
    x = layers.Conv2D(filters=32, kernel_size=3, strides=2, activation="relu")(inputs)
    x = layers.Conv2D(filters=64, kernel_size=3, strides=2, activation="relu")(x)
    x = layers.Conv2D(filters=128, kernel_size=3, strides=2, activation="relu")(x)
    return x

In [ ]:
covnet_block = basic_covnet_block()
x = layers.Flatten()(covnet_block)
x = layers.Dense(1)(x)
model = keras.Model(inputs=inputs, outputs=x)
model.summary()

In [ ]:
model.compile(loss="mse", optimizer="adam", metrics=["mae"])

In [ ]:
k_fold_validation(model, 'basic_covnet_(32, 64,128)_NaN.h5')

## Modelo con VGG16 como extractor de features

In [ ]:
def base_vgg_16_layers(input):
    covnet = keras.applications.vgg16.VGG16(
        include_top=False,
        weights='imagenet',
        input_shape=img_size + (img_channels,))(covnet)
    covnet.trainable = False
    return keras.Model(inputs=input, outputs=covnet)


def output_vgg_16_layers(covnet):
    output = basic_dense_block(covnet, [])
    return keras.Model(inputs=covnet, outputs=output)



In [ ]:
def activate_fine_tuning_on_vgg_16(ft_model):
    # Flag to indicate whether the layers should be trainable
    set_trainable = False

    # Assuming 'vgg16' is the name of the nested VGG16 model
    vgg16 = ft_model.get_layer('vgg16')
    vgg16.trainable = True

    for layer in vgg16.layers:
        # Start fine-tuning from 'block5_conv1'
        if layer.name == 'block5_conv1':
            set_trainable = True

        # Set the trainable flag for the layers
        if set_trainable:
            print(f'Unfreezing layer {layer.name}')
            layer.trainable = True
        else:
            print(f'Freezing layer {layer.name}')
            layer.trainable = False

In [ ]:
def vgg_16_k_fold_validation(model_filename):
    callbacks_list = [
        #    keras.callbacks.EarlyStopping(
        #        monitor="val_loss", patience=4
        #    ),
        keras.callbacks.ModelCheckpoint(
            filepath=model_filename,
            monitor="val_loss",
            save_best_only=True
        ),
        #    keras.callbacks.TensorBoard()
    ]

    i_kf = KFold(n_splits=5, shuffle=True, random_state=2)

    history_store = []

    for i_result in i_kf.split(labels_df):
        i_train = labels_df.iloc[i_result[0]]
        i_test = labels_df.iloc[i_result[1]]

        if debug:
            print(i_result[0])
            print(i_result[1])

            display(labels_df)
            display(i_train)
            print(f'Train size: {len(i_train)}')
            display(i_test)
            print(f'Test size: {len(i_test)}')

        (train_generator, validation_generator) = training_utils.load_generators(i_train, i_test, dataset_path,
                                                                                 batch_size, img_size)

        conv_base = base_vgg_16_layers(keras.layers.Input(shape=img_size + (img_channels,)))

        feature_train = conv_base.predict(train_generator, 2000, verbose=1)
        feature_val = conv_base.predict(validation_generator, 2000, verbose=1)

        feature_train = np.array(feature_train)
        feature_val = np.array(feature_val)

        if debug:
            print(feature_train.shape)
            display(feature_train[0][0])
            print(i_train['count'].values)

        i_model = output_vgg_16_layers(keras.Input(shape=feature_train.shape[1:]))

        i_model.compile(loss="mse", optimizer="adam", metrics=["mae"])

        i_history = [i_model.fit(feature_train, i_train['count'].values,
                                 epochs=epochs,
                                 callbacks=callbacks_list,
                                 validation_data=(feature_val, i_test['count'].values),
                                 ), ]

        merged_model = keras.Model(inputs=conv_base.input, outputs=i_model(conv_base.output))
        activate_fine_tuning_on_vgg_16(merged_model)
        merged_model.compile(loss="mse", optimizer="adam", metrics=["mae"])

        merged_model.summary()

        i_history.append(merged_model.fit(train_generator,
                                          epochs=epochs,
                                          callbacks=callbacks_list,
                                          validation_data=validation_generator,
                                          ))

        history_store.append(i_history)

    return history_store
